<a href="https://colab.research.google.com/github/leah-apking/homework/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install & Import Dependencies

In [1]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-s4ftvaan
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-s4ftvaan
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=ab4b47967fb0e246e069a48a5507500bcd40bf5f73bdbf9bd0345f2d6ad74029
  Stored in directory: /tmp/pip-ephem-wheel-cache-y9fwn78o/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.6 MB/s eta 0:00:00


In [3]:
!python -m spacy download en_core_web_sm

2023-10-26 20:16:10.662043: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 20:16:10.662123: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 20:16:10.665757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 20:16:10.693534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 20:16:13.440547: W tensorflow/compiler/

In [4]:
import pandas as pd
import numpy as np
import pke
from transformers import pipeline

## Pull NPS Feedback Data

In [7]:
month=  '10'
year = '22'
nps_path = f"20{year}.{month}ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [17]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps0 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1 = nps0.fillna(0)
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == 0 else 'True')

In [19]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments_df = comments.dropna()
comments.head()

,LoanNumber,Comments
1,1222644714,Absolutely love UWM!
5,1222586058,I am happy with the overall handling of this f...
7,1222543928,Always fast and efficient. Had some hiccups on...
10,1522129320,Everything went smooth and the UW was outstand...
11,1522128772,Excellent job


In [20]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_short = comments.loc[(comments['WordCount'] > 0) & (comments['WordCount'] < 10)]
nps_short.count()

<ipython-input-20-a644941f0a86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    341
Comments      341
WordCount     341
dtype: int64

In [21]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_med = comments.loc[(comments['WordCount'] >= 10) & (comments['WordCount'] <= 30)]
nps_med.count()

<ipython-input-21-840eeb0e71f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    317
Comments      317
WordCount     317
dtype: int64

In [22]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_long = comments.loc[(comments['WordCount'] > 30) & (comments['WordCount'] <= 400)]
nps_long.count()

<ipython-input-22-906bb5f5c8cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    202
Comments      202
WordCount     202
dtype: int64

In [23]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_xl = comments.loc[comments['WordCount'] > 400]
nps_xl.count()

<ipython-input-23-593fb9c2164a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    1
Comments      1
WordCount     1
dtype: int64

## Extract Sentiment and KeyPhrases from Comments

### "Short" Comments (WordCount < 10)

In [24]:
def get_sentiment(text):

  sentiment_pipeline = pipeline(task = 'sentiment-analysis',
                     model = 'nlptown/bert-base-multilingual-uncased-sentiment',
                     tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

  return sentiment_pipeline(text)

def pke_extract(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  return extractor.get_n_best(n=n, stemming=False)

n=2
nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)

nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)

<ipython-input-24-3ef077a83b70>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)


<ipython-input-24-3ef077a83b70>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)


In [25]:
nps_short['Keywords'] = nps_short['Keywords'].astype(str)

def clean_keywords(keywords_str):
  keywords_list = eval(keywords_str)
  return [keyword for keyword, _ in keywords_list]

nps_short['Keywords'] = nps_short['Keywords'].apply(clean_keywords)

<ipython-input-25-367733debbc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Keywords'].astype(str)
<ipython-input-25-367733debbc6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Keywords'].apply(clean_keywords)


In [26]:
nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)

def clean_sentiment(sentiment_str):
  sentiment_list = eval(sentiment_str)
  return sentiment_list[0]['label'].split(' ')[0]

nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)

<ipython-input-26-e2e120fc2df2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)
<ipython-input-26-e2e120fc2df2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)


In [28]:
nps_short = nps_short.drop(columns={'Keyword', 'Keyword1'})

nps_short.head(15)

,LoanNumber,Comments,WordCount,Keywords,Sentiment
1,1222644714,Absolutely love UWM!,3,[uwm],5
10,1522129320,Everything went smooth and the UW was outstand...,8,"[outstanding, smooth]",5
11,1522128772,Excellent job,2,[excellent job],5
13,1222599887,Underwriters were great on this file!,6,"[great, underwriters]",5
16,1222552345,"100%, make it a smooth an easy process",8,"[easy process, smooth]",5
17,1222607844,Always a great loan with UWM :-),7,"[great loan, uwm]",5
23,1222592512,Great gob!,2,[great gob],5
33,1222584979,Thank you for being awesome ??,6,[awesome],5
46,1522129153,Great process all the way around,6,"[great process, way]",5
66,1222555853,Thank you perfect job,4,[perfect job],5


### Medium Comments (WordCount 10-30)

In [29]:
# def pke_extract5(text):

#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   return extractor.get_n_best(n=5, stemming=False)

n=4
nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract)

nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)

<ipython-input-29-f88018c8d639>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract)
<ipython-input-29-f88018c8d639>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)


In [30]:
nps_med['Keywords'] = nps_med['Keywords'].astype(str)
nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)

nps_med['Keywords'] = nps_med['Keywords'].apply(clean_keywords)
nps_med['Sentiment'] = nps_med['Sentiment'].apply(clean_sentiment)

<ipython-input-30-ce786f379d17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Keywords'].astype(str)
<ipython-input-30-ce786f379d17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)
<ipython-input-30-ce786f379d17>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
# #split column of lists into two new columns
# split = pd.DataFrame(nps_med['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3'])

# #join split columns back to original DataFrame
# nps_med = pd.concat([nps_med, split], axis=1)

In [31]:
nps_med.head()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
5,1222586058,I am happy with the overall handling of this f...,12,"[overall handling, file, happy]",5
7,1222543928,Always fast and efficient. Had some hiccups on...,20,"[uwm, best, hiccups, efficient]",5
27,1222509794,Very quick and smooth transaction! Underwriter...,18,"[underwriters, closers, smooth transaction, pr...",5
28,1222610373,Shout out to Alex Harrison. He is always respo...,30,"[uw11, knowledgeable, responsive, file]",5
32,1522112456,AE and ability to communicate instantly with o...,12,"[ability, outstanding service]",5


### Long Comments (WordCount 30-400)

In [32]:
# def get_sentiment(text):

#   sentiment_pipeline = pipeline(task = 'sentiment-analysis',
#                      model = 'nlptown/bert-base-multilingual-uncased-sentiment',
#                      tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

#   return sentiment_pipeline(text)

# def pke_extract8(text):

#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   return extractor.get_n_best(n=8, stemming=False)

n = 8
nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract)

nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)

<ipython-input-32-7479e77bcc86>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract)
<ipython-input-32-7479e77bcc86>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)


In [33]:
nps_long['Keywords'] = nps_long['Keywords'].astype(str)
nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)

nps_long['Keywords'] = nps_long['Keywords'].apply(clean_keywords)
nps_long['Sentiment'] = nps_long['Sentiment'].apply(clean_sentiment)

<ipython-input-33-3aba88d1555d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Keywords'].astype(str)
<ipython-input-33-3aba88d1555d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)
<ipython-input-33-3aba88d1555d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [ ]:
# #split column of lists into two new columns
# split = pd.DataFrame(nps_long['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3', 'Keyword4', 'Keyword5'])

# #join split columns back to original DataFrame
# nps_long = pd.concat([nps_long, split], axis=1)

## Complete DataFrame

In [ ]:

nps_short.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
1217,1.223462e+09,Don't add conditions after clear to close is i...,9.0,[clear],1
1227,1.523112e+09,slow process,2.0,[slow process],2
1231,1.523112e+09,slow process,2.0,[slow process],2
1245,1.223502e+09,unnecessary touches,2.0,[unnecessary touches],1
1257,1.223527e+09,The ROV process needs an overhaul,6.0,[overhaul],3


,0
6,NaN
8,NaN
28,NaN
32,NaN
54,NaN
...,...
1232,NaN
1234,NaN
1260,NaN
1262,NaN


In [34]:
concat1 = pd.concat([nps_short, nps_med])
concat2 = pd.concat([concat1, nps_long])
concat2.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
2536,0,"Nothing at this point, but UWM could either le...",72,"[uwm, secret overlays, guidelines, list, speci...",2
2561,0,Tyler Webb did an excellent job on this file a...,36,"[nice job tyler thanks, deadline, excellent jo...",5
2564,0,Thanks to haas (AE) who explained how the appr...,47,"[haas, appraisal credit works, excited, thanks...",1
2570,0,Everyone is so nice and helpful as this was my...,40,"[hours, title company, clients, helpful, nice,...",4
2576,0,The AE was there to help from start to finish ...,42,"[great, start, underwriter, deal, communicatio...",5


In [35]:
merged = pd.merge(concat2, nps1, on='LoanNumber')
merged1 = merged.drop(columns={'Comments_y', 'Comment', 'NetPromoterLabel'})
merged2 = merged1.rename(columns={'Comments_x': 'Comments'})
merged2.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment,ClientName,CompanyName,Rating,TeamMemberName
21581,1222674721,The underwriter went to great links to make th...,43,"[underwriter, absolute nightmare, start, file,...",1,Susan Hooker,"Lincoln Lending Group, Inc.",0.0,Justin Shaw
21582,1222731687,Your closing department needs help. Seriously ...,49,"[hardest lender, help, uwm, loan, closing depa...",2,Brady William Bell,AK Bell Inc.,0.0,Aaron Kochut
21583,1222633224,Terrible underwrite regarding income calculati...,70,"[replacement guarantee insurance, non-agency, ...",1,Gerald Lynn Greer,"Mortgage Only, Inc.",0.0,Hunter Jackson
21584,1222681252,Several issues with UW signing off on conditio...,40,"[docs, closing, major problems, hours, conditi...",1,Jon Franklin Traver,"NEXA Mortgage, LLC C",0.0,Mahb Rahman
21585,1222651492,This was a purchase closing which did not clos...,131,"[matter, potential decline, business, borrower...",1,Justo Sepulveda,"USA Mortgage Network, Inc.",0.0,Mario Alias


In [36]:
from google.colab import files
merged2.to_csv('CommentsProcessed_2022.10.csv')
files.download('CommentsProcessed_2022.10.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# final_df = merged2.groupby(merged2['LoanNumber']).first()

In [ ]:
nps_short = nps_short.drop(columns={'Keywords'})
nps_med = nps_med.drop(columns={'Keywords'})
nps_long = nps_long.drop(columns={'Keywords'})

In [ ]:
from google.colab import files
nps_short.to_csv('CommentsProcessed4.csv', encoding = 'utf-8-sig')
files.download('CommentsProcessed4.csv')

# nps_med.to_csv('CommentsProcessed2.csv', encoding = 'utf-8-sig')
# files.download('CommentsProcessed2.csv')

# nps_long.to_csv('CommentsProcessed3.csv', encoding = 'utf-8-sig')
# files.download('CommentsProcessed3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Last week

In [ ]:
sentimentlist1 = []

ranklist1 = []
phraselist1= []
# scorelist1 = []

def sentiment1(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):

    sentimentlist1.append(data)

def pke_extract1(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases1 = extractor.get_n_best(n=1, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases1):

    ranklist1.append(i)
    phraselist1.append(candidate)
    # scorelist1.append(score)

for n in range(len(nps_short)):
    text = nps_short.iloc[n,1]
    sentiment1(text)
    pke_extract1(text)



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

In [ ]:
# words1 = pd.DataFrame(list(zip(phraselist1, ranklist1)), columns = ['1st', 'Rank'])
# words1

,1st,Score
0,loans,0
1,support,0
2,great job,0
3,account executive,0
4,great job,0
5,speed,0
6,cheers,0
7,closing,0
8,clean,0
9,ae beau,0


## Build DataFrame

In [ ]:
df1 = pd.DataFrame(sentimentlist1)
df3 = pd.DataFrame(sentimentlist3)
df5 = pd.DataFrame(sentimentlist5)


,label,score
0,POSITIVE,0.999392
1,POSITIVE,0.999794
2,POSITIVE,0.999848
3,POSITIVE,0.999876
4,POSITIVE,0.999876
...,...,...
127,NEGATIVE,0.998263
128,NEGATIVE,0.953232
129,NEGATIVE,0.992574
130,NEGATIVE,0.988391


In [ ]:
words1 = pd.DataFrame(list(zip(phraselist, scorelist)), columns = ['1st', 'Score'])
words1

,1st,Score
0,pleasure,0.393617
1,excellent team work,1.000000
2,amazing,0.500000
3,speedy,0.393617
4,fast process,0.408888
...,...,...
103,jumbos,0.284205
104,closing day,0.500000
105,better communication,0.500000
106,clear,0.500000


In [ ]:
def divide_chunks1(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 1

phrases = list(divide_chunks1(phraselist, n))
columns = ['1st']
keywords_df1 = pd.DataFrame(phrases,columns=columns)
keywords_df1

,1st
0,pleasure
1,great love uwm
2,excellent team work
3,amazing
4,speedy
...,...
150,clear
151,slow process
152,slow process
153,unnecessary touches


In [ ]:
def divide_chunks3(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 3

phrases = list(divide_chunks3(phraselist, n))
columns = ['1st', '2nd', '3rd']
keywords_df3 = pd.DataFrame(phrases,columns=columns)
keywords_df3

In [ ]:
def divide_chunks5(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks5(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
keywords_df5 = pd.DataFrame(phrases,columns=columns)
keywords_df5

,1st,2nd,3rd,4th,5th
0,uwm,closing team,awesome,easy,process
1,christina plante,jackie berry,awesome,hiccups,couple
2,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,garett hess,conditions,succinct,gentleman,prompt
4,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...
228,fault,loan,disaster,ratings,mistake
229,file,unhelpful,dismissive,complete,rate
230,email,dollars,closing,consistent uw,nice
231,surprise,enote,uwm,agent,builder


In [ ]:
# phrases2 = list(divide_chunks(phraselist2, n))

# keywords2_df = pd.DataFrame(phrases2,columns=columns)
# keywords2_df

In [ ]:
len(scorelist)

155

In [ ]:
df1 = pd.concat([sentiment_df, keywords_df], axis=1)
# df2 = pd.concat([df1, nps_comments], axis=1)
# df2.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   263 non-null    object 
 1   score   263 non-null    float64
 2   1st     233 non-null    object 
 3   2nd     233 non-null    object 
 4   3rd     232 non-null    object 
 5   4th     232 non-null    object 
 6   5th     232 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.5+ KB


In [ ]:
df1

,label,score,1st,2nd,3rd,4th,5th
0,POSITIVE,0.832655,uwm,closing team,awesome,easy,process
1,POSITIVE,0.999789,christina plante,jackie berry,awesome,hiccups,couple
2,POSITIVE,0.912482,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,POSITIVE,0.999395,garett hess,conditions,succinct,gentleman,prompt
4,NEGATIVE,0.851754,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...,...,...
258,NEGATIVE,0.999366,NaN,NaN,NaN,NaN,NaN
259,NEGATIVE,0.997779,NaN,NaN,NaN,NaN,NaN
260,NEGATIVE,0.962565,NaN,NaN,NaN,NaN,NaN
261,NEGATIVE,0.998787,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import files

export_path = f"{month}_NPS_Comments_Processed.csv"
comments.to_csv(export_path, encoding = 'utf-8-sig')
files.download(export_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>